In [1]:
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

/home/gokkulnath/anaconda3/envs/Keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import keras
from keras.models import Sequential,Model
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.layers import Dense,Conv2D,MaxPool2D,Dropout,BatchNormalization,Activation,Merge,Flatten
from keras.initializers import TruncatedNormal
from keras.losses import categorical_crossentropy
from keras.preprocessing import image
from keras.utils import to_categorical


import numpy as np
from scipy.io import loadmat
import h5py

Using TensorFlow backend.


In [3]:
# Mnist Data Preparation :

(X_train,Y_train),(X_test,Y_test)=mnist.load_data()

In [4]:
# Filter of Data with Labels 5-9
X_train_fil=[]
Y_train_fil=[]
for i in range(len(X_train)):
    if Y_train[i]>=5 and Y_train[i]<=9:
        X_train_fil.append(X_train[i])
        Y_train_fil.append(Y_train[i])
        

In [5]:
X_test_fil=[]
Y_test_fil=[]
for i in range(len(X_test)):
    if Y_test[i]>=5 and Y_test[i]<=9:
        X_test_fil.append(X_test[i])
        Y_test_fil.append(Y_test[i])
        

In [6]:
X_train_fil=np.asarray(X_train_fil)
Y_train_fil=np.asarray(Y_train_fil)

X_test_fil=np.asarray(X_test_fil)
Y_test_fil=np.asarray(Y_test_fil)

# Subtact to make 5 neurons at the last layer
Y_train_fil1=Y_train_fil-5
Y_test_fil1=Y_test_fil-5


Y_train_fil_cat=to_categorical(Y_train_fil1)
Y_test_fil_cat=to_categorical(Y_train_fil1)

In [7]:
print("Training Data", X_train_fil.shape,"Trainning Label",Y_train_fil_cat.shape)
print("Test Data", X_test_fil.shape,"Test Label",Y_test_fil_cat.shape)


Training Data (29404, 28, 28) Trainning Label (29404, 5)
Test Data (4861, 28, 28) Test Label (29404, 5)


In [8]:
PathtoSVNH='data/SVNH/'
SVNH_tr=loadmat(PathtoSVNH+'train_32x32.mat')
SVNH_tst=loadmat(PathtoSVNH+'test_32x32.mat')

In [9]:
SVNH_train=SVNH_tr['X']
SVNH_train_label=SVNH_tr['y']
SVNH_test=SVNH_tst['X']
SVNH_test_label=SVNH_tst['y']

SVNH_train=SVNH_train.swapaxes(0,3).swapaxes(2,3)
SVNH_test=SVNH_test.swapaxes(0,3).swapaxes(2,3)

SVNH_train_label[SVNH_train_label ==10] = 0

SVNH_test_label[SVNH_test_label ==10] = 0

SVNH_train_label_cat=to_categorical(SVNH_train_label)

SVNH_test_label_cat=to_categorical(SVNH_test_label)

In [10]:
print("Training Data", SVNH_train.shape,"Trainning Label",SVNH_train_label_cat.shape)
print("Test Data", SVNH_test.shape,"Test Label",SVNH_test_label_cat.shape)


Training Data (73257, 32, 32, 3) Trainning Label (73257, 10)
Test Data (26032, 32, 32, 3) Test Label (26032, 10)


In [14]:
SVNH_test_fil=[]
SVNH_test_label_fil=[]
for i in range(SVNH_test.shape[0]):
    if (SVNH_test_label[i])>=0 and (SVNH_test_label[i])<=4:
        SVNH_test_fil.append(SVNH_test[i,:,:,:])
        SVNH_test_label_fil.append(SVNH_test_label[i][0])

SVNH_train_fil=[]
SVNH_train_label_fil=[]
for i in range(SVNH_train.shape[0]):
    if ((SVNH_train_label[i][0])>=0 and (SVNH_train_label[i][0])<=4):
        SVNH_train_fil.append(SVNH_train[i,:,:,:])
        SVNH_train_label_fil.append(SVNH_train_label[i][0])

In [15]:
SVNH_train_fil=np.asarray(SVNH_train_fil)
SVNH_train_label_fil=to_categorical(np.asarray(SVNH_train_label_fil))

SVNH_test_fil=np.asarray(SVNH_test_fil)
SVNH_test_label_fil=to_categorical(np.asarray(SVNH_test_label_fil))


In [16]:
print("Training Data", SVNH_train_fil.shape,"Trainning Label",SVNH_train_label_fil.shape)
print("Test Data", SVNH_test_fil.shape,"Test Label",SVNH_test_label_fil.shape)

Training Data (45349, 32, 32, 3) Trainning Label (45349, 5)
Test Data (16397, 32, 32, 3) Test Label (16397, 5)


# Perform Center Crop

In [17]:
#SVNH_train_fil=SVNH_train_fil[:,2:30,2:30,:]
#SVNH_test_fil=SVNH_test_fil[:,2:30,2:30,:]

SVNH_train_fil=SVNH_train[:,2:30,2:30,:]
SVNH_test_fil=SVNH_test[:,2:30,2:30,:]


In [18]:
print("Training Data", SVNH_train_fil.shape,"Trainning Label",SVNH_train_label_cat.shape)
print("Test Data", SVNH_test_fil.shape,"Test Label",SVNH_test_label_cat.shape)

Training Data (73257, 28, 28, 3) Trainning Label (73257, 10)
Test Data (26032, 28, 28, 3) Test Label (26032, 10)


In [20]:
#Use Lenet Architecture and Perform Domain Transfer Learning between SVNH 0-4 to MNIST 5-9

Base_Model=Sequential()
Base_Model.add(Conv2D(batch_input_shape=(None,28,28,3),kernel_size=[5,5],filters=6))
Base_Model.add(MaxPool2D(pool_size=2, strides=2, padding='valid'))
Base_Model.add(Conv2D(kernel_size=[5,5],filters=16))
Base_Model.add(MaxPool2D(pool_size=2,strides=2,padding='valid'))
Base_Model.add(Flatten())
Base_Model.add(Dense(units=120,activation='sigmoid'))
Base_Model.add(Dense(units=84,activation='sigmoid'))
Base_Model.add(Dense(units=10,activation='softmax'))

In [21]:
Base_Model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 6)         456       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 6)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 16)          2416      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               30840     
_________________________________________________________________
dense_2 (Dense)              (None, 84)                10164     
__________

In [22]:
adam=keras.optimizers.Adam(lr=0.0001, beta_1=0.75, beta_2=0.99, epsilon=None, decay=0.0)


In [23]:
#keras.callbacks.TensorBoard(log_dir='/Graph', histogram_freq=0,write_graph=True, write_images=True)
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)


In [24]:
Base_Model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [25]:
Base_Model.fit(SVNH_train_fil, SVNH_train_label_cat,batch_size=64,epochs=10,verbose=1,validation_data=(SVNH_test_fil, SVNH_test_label_cat),callbacks=[tbCallBack])

Train on 73257 samples, validate on 26032 samples
Epoch 1/10
73257/73257 [==============================] - 5s 64us/step - loss: 2.2118 - acc: 0.1919 - val_loss: 2.1371 - val_acc: 0.2409
Epoch 2/10
73257/73257 [==============================] - 3s 43us/step - loss: 2.0265 - acc: 0.3095 - val_loss: 1.8632 - val_acc: 0.4100
Epoch 3/10
73257/73257 [==============================] - 3s 43us/step - loss: 1.6898 - acc: 0.4523 - val_loss: 1.5205 - val_acc: 0.4967
Epoch 4/10
73257/73257 [==============================] - 3s 43us/step - loss: 1.3907 - acc: 0.5574 - val_loss: 1.2881 - val_acc: 0.5889
Epoch 5/10
73257/73257 [==============================] - 3s 44us/step - loss: 1.1761 - acc: 0.6389 - val_loss: 1.1030 - val_acc: 0.6591
Epoch 6/10
73257/73257 [==============================] - 3s 43us/step - loss: 1.0178 - acc: 0.6960 - val_loss: 0.9953 - val_acc: 0.7059
Epoch 7/10
73257/73257 [==============================] - 3s 44us/step - loss: 0.9000 - acc: 0.7338 - val_loss: 0.8928 - val_acc

In [26]:
Base_Model.fit(SVNH_train_fil, SVNH_train_label_cat,batch_size=64,epochs=10,verbose=1,validation_data=(SVNH_test_fil, SVNH_test_label_cat),callbacks=[tbCallBack])

Train on 73257 samples, validate on 26032 samples
Epoch 1/10
73257/73257 [==============================] - 3s 43us/step - loss: 0.6409 - acc: 0.8099 - val_loss: 0.7019 - val_acc: 0.7881
Epoch 2/10
73257/73257 [==============================] - 3s 43us/step - loss: 0.6058 - acc: 0.8196 - val_loss: 0.6865 - val_acc: 0.7908
Epoch 3/10
73257/73257 [==============================] - 3s 45us/step - loss: 0.5786 - acc: 0.8275 - val_loss: 0.6566 - val_acc: 0.8016
Epoch 4/10
73257/73257 [==============================] - 3s 44us/step - loss: 0.5549 - acc: 0.8349 - val_loss: 0.6423 - val_acc: 0.8066
Epoch 5/10
73257/73257 [==============================] - 3s 43us/step - loss: 0.5361 - acc: 0.8407 - val_loss: 0.6278 - val_acc: 0.8105
Epoch 6/10
73257/73257 [==============================] - 3s 44us/step - loss: 0.5184 - acc: 0.8458 - val_loss: 0.6235 - val_acc: 0.8124
Epoch 7/10
73257/73257 [==============================] - 3s 44us/step - loss: 0.5050 - acc: 0.8508 - val_loss: 0.6134 - val_acc

In [31]:
adam=keras.optimizers.Adam(lr=0.0001, beta_1=0.75, beta_2=0.99, epsilon=None, decay=0.0)
Base_Model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [32]:
Base_Model.fit(SVNH_train_fil, SVNH_train_label_cat,batch_size=64,epochs=10,verbose=1,validation_data=(SVNH_test_fil, SVNH_test_label_cat),callbacks=[tbCallBack])

Train on 73257 samples, validate on 26032 samples
Epoch 1/10
73257/73257 [==============================] - 3s 42us/step - loss: 0.2784 - acc: 0.9192 - val_loss: 0.6086 - val_acc: 0.8303
Epoch 2/10
73257/73257 [==============================] - 3s 45us/step - loss: 0.2801 - acc: 0.9189 - val_loss: 0.6157 - val_acc: 0.8259
Epoch 3/10
73257/73257 [==============================] - 3s 47us/step - loss: 0.2770 - acc: 0.9192 - val_loss: 0.6085 - val_acc: 0.8288
Epoch 4/10
73257/73257 [==============================] - 3s 43us/step - loss: 0.2771 - acc: 0.9192 - val_loss: 0.6047 - val_acc: 0.8302
Epoch 5/10
73257/73257 [==============================] - 3s 42us/step - loss: 0.2750 - acc: 0.9199 - val_loss: 0.6105 - val_acc: 0.8306
Epoch 6/10
73257/73257 [==============================] - 3s 45us/step - loss: 0.2739 - acc: 0.9201 - val_loss: 0.6121 - val_acc: 0.8294
Epoch 7/10
73257/73257 [==============================] - 3s 44us/step - loss: 0.2726 - acc: 0.9211 - val_loss: 0.6127 - val_acc

In [28]:
Base_Model.load_weights('SVNH_model_1.h5')

In [ ]:
Base_Model.save('SVNH_model_1.h5')
#model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'


In [35]:
adam=keras.optimizers.Adam(lr=1e-6, beta_1=0.75, beta_2=0.99, epsilon=None, decay=0.0)


In [ ]:
Base_Model.fit(SVNH_train_fil, SVNH_train_label_cat,batch_size=64,epochs=10,verbose=1,validation_data=(SVNH_test_fil, SVNH_test_label_cat),callbacks=[tbCallBack])

Train on 73257 samples, validate on 26032 samples
Epoch 1/10
73257/73257 [==============================] - 3s 43us/step - loss: 0.2526 - acc: 0.9277 - val_loss: 0.6193 - val_acc: 0.8294
Epoch 2/10
73257/73257 [==============================] - 3s 43us/step - loss: 0.2517 - acc: 0.9279 - val_loss: 0.6251 - val_acc: 0.8292
Epoch 3/10
73257/73257 [==============================] - 3s 44us/step - loss: 0.2499 - acc: 0.9281 - val_loss: 0.6225 - val_acc: 0.8305
Epoch 4/10
73257/73257 [==============================] - 3s 44us/step - loss: 0.2486 - acc: 0.9289 - val_loss: 0.6257 - val_acc: 0.8294
Epoch 5/10
 3968/73257 [>.............................] - ETA: 2s - loss: 0.2352 - acc: 0.9330

In [34]:
tf.__version__

'1.4.1'